In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

url = 'https://employment.en-japan.com/wish/search_list/?companytype=0&worktype=0&areaid=23_24_21_50&occupation=101000_102500_103000_103500_104500_105000_105500_109000&indexNoWishArea=0&sort=wish'
r = requests.get(url)
r.raise_for_status()

soup = BeautifulSoup(r.content, 'lxml')
jobs = soup.find_all('div', class_='jobNameArea')

deta = []
next_page_flag = True
next_page_num = 2

while next_page_flag:
    for job in jobs:
        company_name = job.find('span', class_='company').text
        page_url = 'https://employment.en-japan.com' + job.find('a').get('href')

        detail_r = requests.get(page_url)
        detail_r.raise_for_status()

        detail_soup = BeautifulSoup(detail_r.content, 'lxml')

        sleep(1)
        if(job.find('a').get('class')[0] == 'job'):
            company_url = detail_soup.find('a', class_='previewOption').text
            detam = {
                '会社名': company_name,
                '企業ホームページ': company_url
            }
        else:
            page_url2 = detail_soup.find('iframe').get('src')
            detail_r2 = requests.get(page_url2)
            detail_r2.raise_for_status()

            sub_soup = BeautifulSoup(detail_r2.content, 'lxml')
            com_table = sub_soup.find('table', class_='companyTable')
            
            try:
                company_url = com_table.find('a').get('href')
                detam = {
                    '会社名': company_name,
                    '企業ホームページ': company_url
                }
            except AttributeError as e:
                detam = {
                    '会社名': company_name,
                    '企業ホームページ': ''
                }
        deta.append(detam)
    try:
        next_url = url + '&pagenum=' + str(next_page_num)
        next_r = requests.get(next_url)
        next_r.raise_for_status()
        next_soup = BeautifulSoup(next_r.content, 'lxml')
        jobs = next_soup.find_all('div', class_='jobNameArea')
        #print(jobs)
        print(next_page_num)
        next_page_num += 1
        sleep(1)
    except requests.HTTPError as e:
        next_page_flag = False

df = pd.DataFrame(deta).reindex(columns=['会社名', '企業ホームページ'])
# print(df)

df.to_csv('section5.csv', index=None, encoding='utf-8-sig')

2
3
4
5


AttributeError: 'NoneType' object has no attribute 'text'

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver

url = 'https://employment.en-japan.com/wish/search_list/?companytype=0&worktype=0&areaid=23_24_21_50&occupation=101000_102500_103000_103500_104500_105000_105500_109000&indexNoWishArea=0&sort=wish'
r = requests.get(url)
r.raise_for_status()

soup = BeautifulSoup(r.content, 'lxml')
jobs = soup.find_all('div', class_='jobNameArea')

next_page_flag = True
next_page_num = 20
while next_page_flag:
    try:
        next_url = url + '&pagenum=' + str(next_page_num)
        next_r = requests.get(next_url)
        next_r.raise_for_status()
        print(next_page_num)
        next_page_num += 1
        sleep(1)
    except requests.HTTPError as e:
        next_page_flag = False
        print(e)
print('end')
        
    


20
21
22
23
404 Client Error: Not Found for url: https://employment.en-japan.com/wish/search_list/?companytype=0&worktype=0&areaid=23_24_21_50&occupation=101000_102500_103000_103500_104500_105000_105500_109000&indexNoWishArea=0&sort=wish&pagenum=24
end
